# Python dependencies

In [ ]:
%pip install requests
%pip install pandas
%pip install datetime

# Configuration

Settings to be configured per individual. 

TODO: configure these settings outside of the notebook so they don't mess with source control. (environment variables?)

In [ ]:
try:
    with open('.github_token', 'r') as f:
        authtoken = f.read()
except FileNotFoundError:
    # get an auth token using the steps here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token
    authtoken = input('Please enter your GitHub token: ')
    with open('.github_token', 'w') as f:
        f.write(authtoken)

In [ ]:
import os

resultsDownloadLocation = 'c:\\temp\\testResults'
if (not os.path.exists(resultsDownloadLocation)):
    os.makedirs(resultsDownloadLocation)


# Retrieving Data

The github action "Aggregate Test Results" runs daily and collects all the results for the previous day into a single json file.

These steps will:

- Find the last 50 runs (you can increase this if you want to look back further
- Download the artifacts from those runs into memory
- Write the .json file from within the artifact to disk (only if there isn't already an up to date file on disk)
- Load all results into a pandas DataFrame

In [ ]:
import requests


def getRuns():
    runsResponse = requests.get(
        "https://api.github.com/repos/microsoft/vscode-jupyter/actions/workflows/aggregate-test-results.yml/runs?per_page=50",
        headers={
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {authtoken}",
            },   
    )
    
    if runsResponse.status_code != 200:
        print(f"Error {runsResponse.status_code}")
        raise Exception("Error getting runs")

    print(f"Found {len(runsResponse.json()['workflow_runs'])} runs")

    return runsResponse.json()["workflow_runs"]

runs = getRuns()

In [ ]:
from datetime import datetime

alreadyDownloaded = {}
for file in os.listdir(resultsDownloadLocation):
    path = os.path.join(resultsDownloadLocation, file)
    lastModified = datetime.fromtimestamp(os.path.getmtime(path))
    alreadyDownloaded[file] = lastModified

print(f"Already downloaded {len(alreadyDownloaded)} result files, they will be skipped unless there is a newer version")

def shouldDownload(name, timestamp):
    fileDate = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
    if name in alreadyDownloaded:
        if alreadyDownloaded[name] >= fileDate:
            return False
            
    alreadyDownloaded[name] = fileDate
    return True
    

In [ ]:
import io
import json
import zipfile


def getArtifactData(id):
    testResultsResponse = requests.get(
        f"https://api.github.com/repos/microsoft/vscode-jupyter/actions/artifacts/{id}/zip",
        headers={
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {authtoken}",
        },
    )

    if testResultsResponse.status_code != 200:
        print(f"Error {testResultsResponse.status_code} getting artifact {id}")

    return testResultsResponse.content

def saveResultsFile(zipData, timeStamp):
    with zipfile.ZipFile(io.BytesIO(zipData)) as artifact:
        for name in artifact.namelist():
            print(f'checking {name} at {timeStamp}')
            if shouldDownload(name, timeStamp):
                content = artifact.read(name)
                print(f"    saving {name}")
                with open(f'{resultsDownloadLocation}\\{name}', 'wb') as f:
                    f.write(content) 

print(f"Getting artifacts from {len(runs)} runs")
for run in runs:
    artifactUrl = run["artifacts_url"]
    print(f"Getting artifacts from {artifactUrl} from {run['created_at']}")
    artifactsResponse = requests.get(
        artifactUrl, headers={
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {authtoken}",
            }
    )

    if artifactsResponse.status_code != 200:
        print(f"Error {artifactsResponse.status_code} getting artifact {id}")
    else:
        artifacts = artifactsResponse.json()["artifacts"]
        for artifact in artifacts:
            rawData = getArtifactData(artifact["id"])
            testRunResults = saveResultsFile(rawData, run["created_at"])

In [ ]:
from datetime import datetime, timedelta

import pandas as pd

testResults = []
for file in os.listdir(resultsDownloadLocation):
    path = f'{resultsDownloadLocation}\\{file}'
    if datetime.fromtimestamp(os.path.getmtime(path)) < datetime.now() - timedelta(days=50):
        # limit the amount of results we load
        continue

    with open(path, 'r') as f:
        try:
            df = pd.read_json(f)
            testResults.append(df)
        except Exception as e:
            print(f'Error reading {file}: {e}')

df = pd.concat(testResults)
# strip off the time to help grouping, but keep as datetime type
df["datetime"] = pd.to_datetime(df["date"])
df["date"] = pd.to_datetime(df["date"]).dt.date

print(f"{len(df)} test results collected between {df['date'].min()} and {df['date'].max()}")

df.head()

# Reporting

In [ ]:
from datetime import date, timedelta

recentFailures = df[df['date'] > date.today() - timedelta(days=7)]
recentFailures = recentFailures[recentFailures['status'] == 'failed'].dropna()
# recentFailures = recentFailures[recentFailures['scenario'] != 'TestLogs-raw-nonConda-3.10---windows-latest']
recentFailures = recentFailures.groupby(['testName', 'suite']).agg(failureCount=('testName', 'count'))

recentFailures.sort_values(by=['failureCount', 'suite'], ascending=False).head(20)

## Failure of a specific test

In [ ]:
testName= 'Cells from python files and the input box are executed in correct order'

testData = df.where(df['testName'] == testName).dropna()
passes = testData.where(testData['status'] == 'passed').dropna()
fails = testData.where(testData['status'] == 'failed').dropna()
successRate = len(passes) / (len(passes) + len(fails))
print(f"'{testName}' failed {len(fails)} times between {testData['date'].min()} and {testData['date'].max()}")
print(f"Success rate: {successRate}")

testData['fail'] = testData['status'] == 'failed'
testData['pass'] = testData['status'] == 'passed'

passfailcounts = testData.groupby(['date']).sum()

passfailcounts.sort_values(by=['date'], ascending=False).head(15)

# line chart not working
# import matplotlib.pyplot as plt
# ax=testData.plot(kind='line', x='date', y='pass', color='Green')

# ax2=testData.plot(kind='line', x='date', y='fail', secondary_y=True,color='Red', ax=ax)

# ax.set_ylabel('Passes')
# ax2.set_ylabel('Failures')
# plt.tight_layout()
# plt.show()

In [ ]:
failures = testData.where(testData['status'] == 'failed').dropna()
failures = failures[['date', 'status', 'scenario', 'runUrl']].sort_values(by=['date'], ascending=False).head(10)

failureMessage = ''
for index, row in failures.iterrows():
    print(f"{row['date']} - {row['scenario']}\n{row['runUrl']}")
    failureMessage += f"{row['date']} - {row['scenario']}\n{row['runUrl']}"

In [ ]:
import io
from urllib import request


# post to create new github issue
def createIssue(title, body):
    print("Creating issue for " + title)
    url = 'https://api.github.com/repos/microsoft/vscode-jupyter/issues'
    data = {
        'title': title,
        'body': body,
        'labels': ['flaky test']
    }
    headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {authtoken}",
    }
    data = json.dumps(data).encode('utf-8')
    req = request.Request(url, data=data, headers=headers, method='POST')
    response = request.urlopen(req)
    print(response.read().decode('utf-8'))

In [ ]:
import ipywidgets as widgets

chk = widgets.Checkbox(
    value=False,
    description='Create issue on github?',
    disabled=False,
    indent=False
)
display(chk)

In [ ]:
if (chk.value):
    createIssue(f"Test failure: {testName}", failureMessage)